# data

In [1]:
%load_ext autoreload
%autoreload 2
import importlib
from importlib import reload  
  
import monotonic
import utils
from train import execute_model_evaluation
from train_config import FlatACLConfig
import train_config
import data_manager
from data_manager import DatasetManager
import metrics
from dataclasses import replace

reload(utils)
reload(monotonic)
reload(train_config)
%load_ext autoreload
%autoreload 2

data_mgr = DatasetManager(device='cuda')
# data_mgr.load_csv('weather', './weather.csv')
data_mgr.load_csv('weather', './weather.csv', random_feature_num_seed_pair=(10, 1955))
# Select Random Features for 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Original Num Columns: 21, Selected Num Columns: 10
Selected Columns: ['wv (m/s)' 'max. wv (m/s)' 'PAR (�mol/m�/s)' 'VPdef (mbar)'
 'H2OC (mmol/mol)' 'Tlog (degC)' 'Tdew (degC)' 'rain (mm)' 'rho (g/m**3)'
 'VPact (mbar)']

Dataset: weather (csv)
Shape: torch.Size([52616, 10])
Channels: 10
Length: 52616
Source: ./weather.csv

Sample data (first 2 rows):
tensor([[ 1.0200e+00,  1.6000e+00,  0.0000e+00,  8.9000e-01,  5.4900e+00,
          1.1450e+01, -1.3300e+00,  0.0000e+00,  1.2806e+03,  5.5400e+00],
        [ 4.3000e-01,  8.4000e-01,  0.0000e+00,  9.5000e-01,  5.4500e+00,
          1.1510e+01, -1.4400e+00,  0.0000e+00,  1.2803e+03,  5.4900e+00]])


In [18]:
data_mgr.datasets['weather']['df']['OT']

0        428.1
1        428.0
2        427.6
3        430.0
4        432.2
         ...  
52691    433.0
52692    439.6
52693    435.2
52694    433.9
52695    436.5
Name: OT, Length: 52616, dtype: float64

# Exp-Weather

## seq=196 Normal

### EigenACL

#### pred=96

In [2]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=196,
    pred_len=96,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([196, 10]), torch.Size([96, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([96, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 286
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 96, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 286
Validation Batches: 39
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4089, mae: 0.3699, hub

#### pred=196

In [3]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=196,
    pred_len=196,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([196, 10]), torch.Size([196, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([196, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 285
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 196, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 285
Validation Batches: 39
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4713, mae: 0.4166,

#### pred=336

In [6]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=196,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([196, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 284
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 284
Validation Batches: 37
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5187, mae: 0.4515,

#### pred=720

In [7]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=196,
    pred_len=720,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([196, 10]), torch.Size([720, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([720, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 281
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 720, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 281
Validation Batches: 34
Test Batches: 76

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5728, mae: 0.4855,

### Timemixer

#### pred=96

In [5]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=196,
    pred_len=96,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([96, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([96, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 286
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 96, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 286
Validation Batches: 39
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4080, mae: 0.3593, huber: 0.1390, swd: 0.2116, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.5309, mae: 0.3094, huber: 0.1188,

#### pred=196

In [6]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=196,
    pred_len=196,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([196, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([196, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 285
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 196, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 285
Validation Batches: 39
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4794, mae: 0.4117, huber: 0.1691, swd: 0.2252, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.5891, mae: 0.3569, huber: 0.1

#### pred=336

In [7]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=196,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 284
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 284
Validation Batches: 37
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5446, mae: 0.4536, huber: 0.1950, swd: 0.2376, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.6810, mae: 0.4139, huber: 0.1

#### pred=720

In [8]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=196,
    pred_len=720,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([720, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([720, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 281
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 720, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 281
Validation Batches: 34
Test Batches: 76

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.6236, mae: 0.5002, huber: 0.2262, swd: 0.2523, target_std: 0.8455
Epoch [1/50], Val Losses: mse: 0.8134, mae: 0.4916, huber: 0.2

### PatchTST

#### pred=96

In [9]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=196,
    pred_len=96,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([96, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([96, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 286
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 96, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 286
Validation Batches: 39
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4334, mae: 0.3791, huber: 0.1503, swd: 0.2085, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.5579, mae: 0.3228, huber: 0.1274,

#### pred=196

In [10]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=196,
    pred_len=196,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([196, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([196, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 285
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 196, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 285
Validation Batches: 39
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4984, mae: 0.4253, huber: 0.1770, swd: 0.2189, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.6223, mae: 0.3683, huber: 0.1

#### pred=336

In [11]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=196,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 284
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 284
Validation Batches: 37
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5616, mae: 0.4670, huber: 0.2030, swd: 0.2330, target_std: 0.8450
Epoch [1/50], Val Losses: mse: 0.7029, mae: 0.4271, huber: 0.1

#### pred=720

In [12]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=196,
    pred_len=720,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([720, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([720, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 281
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 720, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 281
Validation Batches: 34
Test Batches: 76

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.6403, mae: 0.5129, huber: 0.2338, swd: 0.2487, target_std: 0.8455
Epoch [1/50], Val Losses: mse: 0.8501, mae: 0.5144, huber: 0.2

### DLinear

#### pred=96

In [13]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=196,
    pred_len=96,
    channels=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([96, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([96, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 286
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 96, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 286
Validation Batches: 39
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4048, mae: 0.3646, huber: 0.1383, swd: 0.2241, target_std: 0.8450
Epoch [1/50], Val Losses: mse: 0.5160, mae: 0.3087, huber: 0.1138,

#### pred=196

In [14]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=196,
    pred_len=196,
    channels=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([196, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([196, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 285
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 196, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 285
Validation Batches: 39
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4611, mae: 0.4081, huber: 0.1623, swd: 0.2417, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.5638, mae: 0.3505, huber: 0.1

#### pred=336

In [15]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=196,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 284
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 284
Validation Batches: 37
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5114, mae: 0.4444, huber: 0.1836, swd: 0.2587, target_std: 0.8450
Epoch [1/50], Val Losses: mse: 0.6377, mae: 0.4023, huber: 0.1

#### pred=720

In [16]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=196,
    pred_len=720,
    channels=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([196, 10]), torch.Size([720, 10])
Validation set sample shapes: torch.Size([196, 10]), torch.Size([720, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 281
Batch 0: Data shape torch.Size([128, 196, 10]), Target shape torch.Size([128, 720, 10])

Data Preparation: weather
Sequence Length: 196
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 281
Validation Batches: 34
Test Batches: 76

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5719, mae: 0.4833, huber: 0.2078, swd: 0.2843, target_std: 0.8454
Epoch [1/50], Val Losses: mse: 0.7385, mae: 0.4664, huber: 0.1